# Forecasting total system-level demand in the next hour

## possible feature that correlate with bike demand
  1. weather
  1. type of user 
  1. weekday
  1. season
  1. time of day
  1. location


### first simple ML model (workshop 07)

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LinearRegression
import datetime

In [9]:
df = pd.read_csv("../chicago_2016.csv", parse_dates=["start_time","end_time"])
df.head()

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type
0,2016-03-31 23:53:00,2016-04-01 00:07:00,344,458,Ravenswood Ave & Lawrence Ave,Broadway & Thorndale Ave,155,Subscriber
1,2016-03-31 23:46:00,2016-03-31 23:57:00,128,213,Damen Ave & Chicago Ave,Leavitt St & North Ave,4831,Subscriber
2,2016-03-31 23:42:00,2016-03-31 23:46:00,350,210,Ashland Ave & Chicago Ave,Ashland Ave & Division St,4232,Subscriber
3,2016-03-31 23:37:00,2016-03-31 23:55:00,303,458,Broadway & Cornelia Ave,Broadway & Thorndale Ave,3464,Subscriber
4,2016-03-31 23:33:00,2016-03-31 23:37:00,334,329,Lake Shore Dr & Belmont Ave,Lake Shore Dr & Diversey Pkwy,1750,Subscriber


In [20]:
# hinzufügen der Stunde des Starts
df['start_hour']= df["start_time"].apply(lambda x: x.hour)
# hinzufügen des Wochentags
df['weekday']= df["start_time"].apply(lambda x: x.weekday())
# hinzufügen des Monats
df['month']= df["start_time"].apply(lambda x: x.month)
#. honzufügen der Dauer der Fahrt
df["duration"] = (df["end_time"] - df["start_time"]).astype("timedelta64[m]")
df.head()

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,start_hour,weekday,month,duration
0,2016-03-31 23:53:00,2016-04-01 00:07:00,344,458,Ravenswood Ave & Lawrence Ave,Broadway & Thorndale Ave,155,Subscriber,23,3,3,14.0
1,2016-03-31 23:46:00,2016-03-31 23:57:00,128,213,Damen Ave & Chicago Ave,Leavitt St & North Ave,4831,Subscriber,23,3,3,11.0
2,2016-03-31 23:42:00,2016-03-31 23:46:00,350,210,Ashland Ave & Chicago Ave,Ashland Ave & Division St,4232,Subscriber,23,3,3,4.0
3,2016-03-31 23:37:00,2016-03-31 23:55:00,303,458,Broadway & Cornelia Ave,Broadway & Thorndale Ave,3464,Subscriber,23,3,3,18.0
4,2016-03-31 23:33:00,2016-03-31 23:37:00,334,329,Lake Shore Dr & Belmont Ave,Lake Shore Dr & Diversey Pkwy,1750,Subscriber,23,3,3,4.0


In [21]:
# Methode für die Tageszeit
def get_time_of_day(x):
    if(x >= 6 and x <10):
        return "Morning"
    elif(x >= 10 and x <12):
        return "Mid-morning"
    elif(x >= 12 and x <14):
        return "Noon"
    elif(x >= 14 and x <17):
        return "Afternoon"
    elif(x >= 17 and x <21):
        return "Evening"
    else:
        return "Night"

In [22]:
# hinzufügen der Tageszeit
df["time_of_day"] = df["start_hour"].apply(lambda x: get_time_of_day(x))
df.head()

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,start_hour,weekday,month,duration,time_of_day
0,2016-03-31 23:53:00,2016-04-01 00:07:00,344,458,Ravenswood Ave & Lawrence Ave,Broadway & Thorndale Ave,155,Subscriber,23,3,3,14.0,Night
1,2016-03-31 23:46:00,2016-03-31 23:57:00,128,213,Damen Ave & Chicago Ave,Leavitt St & North Ave,4831,Subscriber,23,3,3,11.0,Night
2,2016-03-31 23:42:00,2016-03-31 23:46:00,350,210,Ashland Ave & Chicago Ave,Ashland Ave & Division St,4232,Subscriber,23,3,3,4.0,Night
3,2016-03-31 23:37:00,2016-03-31 23:55:00,303,458,Broadway & Cornelia Ave,Broadway & Thorndale Ave,3464,Subscriber,23,3,3,18.0,Night
4,2016-03-31 23:33:00,2016-03-31 23:37:00,334,329,Lake Shore Dr & Belmont Ave,Lake Shore Dr & Diversey Pkwy,1750,Subscriber,23,3,3,4.0,Night


In [23]:
# hinzufügen der Season
def get_season(x):
    if(x == 1 or x == 2 or x == 12):
        return "Winter"
    elif(x >= 3 and x <= 5):
        return "Spring"
    elif(x >= 6 and x <= 8):
        return "Sommer"
    else:
        return "Autumn"

df["season"] = df["month"].apply(lambda x: get_season(x))
df.head(3)

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,start_hour,weekday,month,duration,time_of_day,season
0,2016-03-31 23:53:00,2016-04-01 00:07:00,344,458,Ravenswood Ave & Lawrence Ave,Broadway & Thorndale Ave,155,Subscriber,23,3,3,14.0,Night,Spring
1,2016-03-31 23:46:00,2016-03-31 23:57:00,128,213,Damen Ave & Chicago Ave,Leavitt St & North Ave,4831,Subscriber,23,3,3,11.0,Night,Spring
2,2016-03-31 23:42:00,2016-03-31 23:46:00,350,210,Ashland Ave & Chicago Ave,Ashland Ave & Division St,4232,Subscriber,23,3,3,4.0,Night,Spring


In [24]:
# hinzufügen der Wetterdaten
weather = pd.read_csv("../weather_hourly_chicago.csv", parse_dates = ["date_time"])
weather = weather.dropna()
weather["hour"] = weather["date_time"].apply(lambda x:x.hour)
weather["day"] = weather["date_time"].apply(lambda x:x.day)
weather.sort_values(by="date_time")
weather.head()

,date_time,max_temp,min_temp,precip,hour,day
0,2015-01-02 01:00:00,-1.7,-1.7,0.0,1,2
1,2015-01-02 02:00:00,-2.2,-2.2,0.0,2,2
2,2015-01-02 03:00:00,-2.8,-2.8,0.0,3,2
3,2015-01-02 04:00:00,-3.3,-3.3,0.0,4,2
4,2015-01-02 05:00:00,-4.4,-4.4,0.0,5,2


In [25]:
# verbinden mit den Fahrraddaten
df["day"] = df["start_time"].apply(lambda x: x.day)
df.head()

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,start_hour,weekday,month,duration,time_of_day,season,day
0,2016-03-31 23:53:00,2016-04-01 00:07:00,344,458,Ravenswood Ave & Lawrence Ave,Broadway & Thorndale Ave,155,Subscriber,23,3,3,14.0,Night,Spring,31
1,2016-03-31 23:46:00,2016-03-31 23:57:00,128,213,Damen Ave & Chicago Ave,Leavitt St & North Ave,4831,Subscriber,23,3,3,11.0,Night,Spring,31
2,2016-03-31 23:42:00,2016-03-31 23:46:00,350,210,Ashland Ave & Chicago Ave,Ashland Ave & Division St,4232,Subscriber,23,3,3,4.0,Night,Spring,31
3,2016-03-31 23:37:00,2016-03-31 23:55:00,303,458,Broadway & Cornelia Ave,Broadway & Thorndale Ave,3464,Subscriber,23,3,3,18.0,Night,Spring,31
4,2016-03-31 23:33:00,2016-03-31 23:37:00,334,329,Lake Shore Dr & Belmont Ave,Lake Shore Dr & Diversey Pkwy,1750,Subscriber,23,3,3,4.0,Night,Spring,31


In [26]:
df = df.merge(weather[["max_temp","min_temp","precip","hour","day"]],left_on=["day","start_hour"],right_on=["day","hour"])
df.head()

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,start_hour,weekday,month,duration,time_of_day,season,day,max_temp,min_temp,precip,hour
0,2016-03-31 23:53:00,2016-04-01 00:07:00,344,458,Ravenswood Ave & Lawrence Ave,Broadway & Thorndale Ave,155,Subscriber,23,3,3,14.0,Night,Spring,31,1.7,1.7,0.0,23
1,2016-03-31 23:53:00,2016-04-01 00:07:00,344,458,Ravenswood Ave & Lawrence Ave,Broadway & Thorndale Ave,155,Subscriber,23,3,3,14.0,Night,Spring,31,13.9,13.9,0.0,23
2,2016-03-31 23:53:00,2016-04-01 00:07:00,344,458,Ravenswood Ave & Lawrence Ave,Broadway & Thorndale Ave,155,Subscriber,23,3,3,14.0,Night,Spring,31,11.7,11.7,0.0,23
3,2016-03-31 23:53:00,2016-04-01 00:07:00,344,458,Ravenswood Ave & Lawrence Ave,Broadway & Thorndale Ave,155,Subscriber,23,3,3,14.0,Night,Spring,31,28.9,28.9,0.0,23
4,2016-03-31 23:53:00,2016-04-01 00:07:00,344,458,Ravenswood Ave & Lawrence Ave,Broadway & Thorndale Ave,155,Subscriber,23,3,3,14.0,Night,Spring,31,28.9,28.9,0.0,23


In [6]:
df.drop(['end_time', 'end_station_id', "start_station_name", "end_station_name" , "start_hour"], axis=1, inplace=True)

NameError: name 'df' is not defined

In [5]:
df.head()

NameError: name 'df' is not defined